# Parameter Estimation Using NRTL

Jaffer Version

Fitting: R32/emimTF2N using composition objective

# Imports

In [1]:
# Todo: import ConcreteModel from pyomo.environ
from pyomo.environ import ConcreteModel, value

# Todo: import FlowsheetBlock from idaes.core
from idaes.core import FlowsheetBlock

In [2]:
from hfc32_emimtf2n_nrtl_VLE import HFCILParameterBlock
import idaes.logger as idaeslog

In [3]:
import pyomo.contrib.parmest.parmest as parmest
import pandas as pd
import numpy as np
from idaes.core.util.model_statistics import degrees_of_freedom

# Set Up Initialized Model

### Grid Search

In [ ]:
tau21 = np.linspace(-20,20,20)
tau12 = np.linspace(-20,20,20)

for i in range(len(tau21)):
    for j in range(len(tau12)):
        print("Trying tau21:",tau21[i])
        print("Trying tau12:",tau12[j])
        
        try:
            def NRTL_model(data):

                #Todo: Create a ConcreteModel object
                m = ConcreteModel()

                #Todo: Create FlowsheetBlock object
                m.fs = FlowsheetBlock(default={"dynamic": False})


                #Todo: Create a properties parameter object with the following options:
                # "valid_phase": ('Liq', 'Vap')
                # "activity_coeff_model": 'NRTL'
                m.fs.properties = HFCILParameterBlock(default={"valid_phase":
                                                             ('Liq', 'Vap'),
                                                             "activity_coeff_model":
                                                             'NRTL'})
                m.fs.state_block = m.fs.properties.state_block_class(
                    default={"parameters": m.fs.properties,
                             "defined_state": True})


                # Fix the state variables on the state block
                # hint: state variables exist on the state block i.e. on m.fs.state_block

                m.fs.state_block.flow_mol.fix(1)
                m.fs.state_block.temperature.fix(298)
                m.fs.state_block.pressure.fix(101325)
                m.fs.state_block.mole_frac_comp["R32"].fix(0.3)
                m.fs.state_block.mole_frac_comp["emimTf2N"].fix(0.7)

                # Fix NRTL specific parameters. 

                # non-randomness parameter - alpha_ij (set at 0.3, 0 if i=j)
                m.fs.properties.\
                    alpha["R32", "R32"].fix(0)
                m.fs.properties.\
                    alpha["R32", "emimTf2N"].fix(0.3)
                m.fs.properties.\
                    alpha["emimTf2N", "emimTf2N"].fix(0)
                m.fs.properties.\
                    alpha["emimTf2N", "R32"].fix(0.3)

                # binary interaction parameter - tau_ij (0 if i=j, else to be estimated later but fixing to initialize)
                m.fs.properties.\
                    tau["R32", "R32"].fix(0)
                m.fs.properties.\
                    tau["R32", "emimTf2N"].fix(tau12[j])
                m.fs.properties.\
                    tau["emimTf2N", "emimTf2N"].fix(0)
                m.fs.properties.\
                    tau["emimTf2N", "R32"].fix(tau21[i])

                # Initialize the flash unit
                print('Trying Init')
                m.fs.state_block.initialize(outlvl=idaeslog.INFO_HIGH)
                print('Finished Init')

                # Fix at actual temperature
                m.fs.state_block.temperature.fix(float(data["temperature"]))

                # Set bounds on variables to be estimated
                m.fs.properties.\
                    tau["R32", "emimTf2N"].setlb(-20)
                m.fs.properties.\
                    tau["R32", "emimTf2N"].setub(20)

                m.fs.properties.\
                    tau["emimTf2N", "R32"].setlb(-20)
                m.fs.properties.\
                    tau["emimTf2N", "R32"].setub(20)

                # Return initialized flash model
                return m
          
            # Testing the initialized model
            test_data = {"temperature": 298}

            m = NRTL_model(test_data)
            
            print(degrees_of_freedom(m))
            
            print('Worked')
            
        except:
            'Skip'


Trying tau21: -20.0
Trying tau12: -20.0
Trying Init
2022-08-01 15:32:06 [INFO] idaes.init.fs.state_block: Initialization Step 1 unbounded - Iterates diverging; problem might be unbounded..
2022-08-01 15:32:09 [INFO] idaes.init.fs.state_block: Initialization Step 2 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:32:11 [INFO] idaes.init.fs.state_block: Initialization Step 3 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:32:14 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:32:16 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exceeded..
Trying tau21: -20.0
Trying tau12: -17.894736842105264
Trying Init
2022-08-01 15:32:16 [INFO] idaes.init.fs.state_block: Initialization Step 1 unbounded - Iterates diverging; problem might be unbounded..
2022-08-01 15:32:19 [INFO] idaes.init.fs.state_block: Initialization Step 2 maxI

2022-08-01 15:34:05 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:34:08 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exceeded..
Trying tau21: -20.0
Trying tau12: 5.2631578947368425
Trying Init
2022-08-01 15:34:08 [INFO] idaes.init.fs.state_block: Initialization Step 1 unbounded - Iterates diverging; problem might be unbounded..
2022-08-01 15:34:11 [INFO] idaes.init.fs.state_block: Initialization Step 2 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:34:13 [INFO] idaes.init.fs.state_block: Initialization Step 3 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:34:16 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:34:19 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exceeded..
Trying

2022-08-01 15:36:07 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:36:09 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exceeded..
Trying tau21: -17.894736842105264
Trying tau12: -7.368421052631579
Trying Init
2022-08-01 15:36:09 [INFO] idaes.init.fs.state_block: Initialization Step 1 unbounded - Iterates diverging; problem might be unbounded..
2022-08-01 15:36:12 [INFO] idaes.init.fs.state_block: Initialization Step 2 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:36:15 [INFO] idaes.init.fs.state_block: Initialization Step 3 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:36:17 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:36:20 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exc

2022-08-01 15:38:09 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:38:11 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exceeded..
Trying tau21: -17.894736842105264
Trying tau12: 17.89473684210526
Trying Init
2022-08-01 15:38:11 [INFO] idaes.init.fs.state_block: Initialization Step 1 unbounded - Iterates diverging; problem might be unbounded..
2022-08-01 15:38:14 [INFO] idaes.init.fs.state_block: Initialization Step 2 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:38:17 [INFO] idaes.init.fs.state_block: Initialization Step 3 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:38:20 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:38:22 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exce

2022-08-01 15:40:06 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exceeded..
Trying tau21: -15.789473684210527
Trying tau12: -1.05263157894737
Trying Init
2022-08-01 15:40:06 [INFO] idaes.init.fs.state_block: Initialization Step 1 unbounded - Iterates diverging; problem might be unbounded..
2022-08-01 15:40:08 [INFO] idaes.init.fs.state_block: Initialization Step 2 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:40:11 [INFO] idaes.init.fs.state_block: Initialization Step 3 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:40:13 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:40:16 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exceeded..
Trying tau21: -15.789473684210527
Trying tau12: 1.0526315789473664
Trying Init
2022-08-01 15:40:16 [INFO] idaes.init.fs.state

...

2022-08-01 15:43:44 [INFO] idaes.init.fs.state_block: Initialization Step 2 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:43:46 [INFO] idaes.init.fs.state_block: Initialization Step 3 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:43:49 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:43:52 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exceeded..
Trying tau21: -13.68421052631579
Trying tau12: 5.2631578947368425
Trying Init
2022-08-01 15:43:52 [INFO] idaes.init.fs.state_block: Initialization Step 1 unbounded - Iterates diverging; problem might be unbounded..
2022-08-01 15:43:54 [INFO] idaes.init.fs.state_block: Initialization Step 2 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:43:57 [INFO] idaes.init.fs.state_block: Initialization Step 3 maxIterations - Maximum Number of Iterations Exce

2022-08-01 15:45:35 [INFO] idaes.init.fs.state_block: Initialization Step 3 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:45:38 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:45:40 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exceeded..
Trying tau21: -11.578947368421053
Trying tau12: -13.68421052631579
Trying Init
2022-08-01 15:45:40 [INFO] idaes.init.fs.state_block: Initialization Step 1 unbounded - Iterates diverging; problem might be unbounded..
2022-08-01 15:45:42 [INFO] idaes.init.fs.state_block: Initialization Step 2 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:45:45 [INFO] idaes.init.fs.state_block: Initialization Step 3 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:45:47 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exc

2022-08-01 15:47:25 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:47:28 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exceeded..
Trying tau21: -11.578947368421053
Trying tau12: 9.473684210526315
Trying Init
2022-08-01 15:47:28 [INFO] idaes.init.fs.state_block: Initialization Step 1 unbounded - Iterates diverging; problem might be unbounded..
2022-08-01 15:47:30 [INFO] idaes.init.fs.state_block: Initialization Step 2 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:47:33 [INFO] idaes.init.fs.state_block: Initialization Step 3 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:47:35 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:47:37 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exce

2022-08-01 15:49:17 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exceeded..
Trying tau21: -9.473684210526317
Trying tau12: -9.473684210526317
Trying Init
2022-08-01 15:49:17 [INFO] idaes.init.fs.state_block: Initialization Step 1 unbounded - Iterates diverging; problem might be unbounded..
2022-08-01 15:49:19 [INFO] idaes.init.fs.state_block: Initialization Step 2 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:49:22 [INFO] idaes.init.fs.state_block: Initialization Step 3 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:49:24 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exceeded..
2022-08-01 15:49:26 [INFO] idaes.init.fs.state_block: Initialization Step 5 maxIterations - Maximum Number of Iterations Exceeded..
Trying tau21: -9.473684210526317
Trying tau12: -7.368421052631579
Trying Init
2022-08-01 15:49:27 [INFO] idaes.init.fs.state_

2022-08-01 16:16:30 [INFO] idaes.init.fs.state_block: Initialization Step 4 maxIterations - Maximum Number of Iterations Exceeded..


In [ ]:
def NRTL_model(data):
    
    #Todo: Create a ConcreteModel object
    m = ConcreteModel()
    
    #Todo: Create FlowsheetBlock object
    m.fs = FlowsheetBlock(default={"dynamic": False})
    

    #Todo: Create a properties parameter object with the following options:
    # "valid_phase": ('Liq', 'Vap')
    # "activity_coeff_model": 'NRTL'
    m.fs.properties = HFCILParameterBlock(default={"valid_phase":
                                                 ('Liq', 'Vap'),
                                                 "activity_coeff_model":
                                                 'NRTL'})
    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    
    # Fix the state variables on the state block
    # hint: state variables exist on the state block i.e. on m.fs.state_block
    
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(298)
    m.fs.state_block.pressure.fix(101325)
    m.fs.state_block.mole_frac_comp["R32"].fix(0.3)
    m.fs.state_block.mole_frac_comp["emimTf2N"].fix(0.7)

    # Fix NRTL specific parameters. 
    
    # non-randomness parameter - alpha_ij (set at 0.3, 0 if i=j)
    m.fs.properties.\
        alpha["R32", "R32"].fix(0)
    m.fs.properties.\
        alpha["R32", "emimTf2N"].fix(0.3)
    m.fs.properties.\
        alpha["emimTf2N", "emimTf2N"].fix(0)
    m.fs.properties.\
        alpha["emimTf2N", "R32"].fix(0.3)

    # binary interaction parameter - tau_ij (0 if i=j, else to be estimated later but fixing to initialize)
    m.fs.properties.\
        tau["R32", "R32"].fix(0)
    m.fs.properties.\
        tau["R32", "emimTf2N"].fix(15)
    m.fs.properties.\
        tau["emimTf2N", "emimTf2N"].fix(0)
    m.fs.properties.\
        tau["emimTf2N", "R32"].fix(-15)

    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.DEBUG)

    # Fix at actual temperature
    m.fs.state_block.temperature.fix(float(data["temperature"]))

    # Set bounds on variables to be estimated
    m.fs.properties.\
        tau["R32", "emimTf2N"].setlb(-20)
    m.fs.properties.\
        tau["R32", "emimTf2N"].setub(20)

    m.fs.properties.\
        tau["emimTf2N", "R32"].setlb(-20)
    m.fs.properties.\
        tau["emimTf2N", "R32"].setub(20)

    # Return initialized flash model
    return m

In [ ]:
from idaes.core.util.model_statistics import degrees_of_freedom
import pytest

# Testing the initialized model
test_data = {"temperature": 298}

m = NRTL_model(test_data)

# Check that degrees of freedom is 0
assert degrees_of_freedom(m) == 0

# Check for output values
assert value(m.fs.state_block.mole_frac_phase_comp['Liq', 'R32']) == pytest.approx(0.218, abs=1e-2)
assert value(m.fs.state_block.mole_frac_phase_comp['Vap', 'R32']) == pytest.approx(1.0, abs=1e-2)

assert value(m.fs.state_block.mole_frac_phase_comp['Liq', 'emimTf2N']) == pytest.approx(0.782, abs=1e-2)
assert value(m.fs.state_block.mole_frac_phase_comp['Vap', 'emimTf2N']) == pytest.approx(0.0, abs=1e-2)